<a href="https://colab.research.google.com/github/Hidenori24/Signate_colab/blob/main/SMBC_GreenChallenge_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## importとシードなどの設定

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
# 日本語フォントを簡単に使う
!pip -q install japanize-matplotlib
import japanize_matplotlib

# シードの設定
np.random.seed(42)
tf.random.set_seed(42)

## Driveへの接続、データの取得

In [17]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/ML/Signate_1634/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### train/test/sample_submission の取り込み

In [18]:
train_df  = pd.read_csv(data_path + 'train.csv', parse_dates=['time'], index_col='time')
df = train_df
test_df   = pd.read_csv(data_path + 'test.csv', parse_dates=['time'], index_col='time')
sample_submission_df = pd.read_csv(data_path + 'sample_submit.csv', header=None)  # header 無し

### 概要確認

In [19]:
# 概要を確認
print("\n--- 学習データ (train_df) ---")
display(train_df.head())
print(f"shape: {train_df.shape}")

print("\n--- テストデータ (test_df) ---")
display(test_df.head())
print(f"shape: {test_df.shape}")

print("\n--- サンプル提出 (sample_submission_df) ---")
display(sample_submission_df.head())
print(f"shape: {sample_submission_df.shape}")


--- 学習データ (train_df) ---


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,seville_rain_1h,seville_rain_3h,seville_snow_3h,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon,price_actual,item_ID
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,64.02,spain_total
2015-01-02 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,58.46,spain_total
2015-01-02 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,54.70,spain_total
2015-01-02 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,54.91,spain_total
2015-01-02 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,53.07,spain_total


shape: (26280, 92)

--- テストデータ (test_df) ---


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,seville_wind_deg,seville_rain_1h,seville_rain_3h,seville_snow_3h,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon,item_ID
time,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+01:00,279.0,0.0,3927.0,895.0,189.0,230.0,1069.0,1893.0,7104.0,53.0,...,343,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 01:00:00+01:00,282.0,0.0,3948.0,878.0,177.0,1269.0,1058.0,1024.0,7101.0,52.0,...,343,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 02:00:00+01:00,283.0,0.0,3791.0,890.0,175.0,2197.0,1052.0,888.0,7100.0,52.0,...,0,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 03:00:00+01:00,280.0,0.0,3671.0,881.0,175.0,2965.0,1032.0,645.0,7101.0,53.0,...,40,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 04:00:00+01:00,286.0,0.0,3460.0,861.0,173.0,2705.0,1001.0,661.0,7101.0,53.0,...,30,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total


shape: (8760, 91)

--- サンプル提出 (sample_submission_df) ---


,0,1
0,2018-01-01 00:00:00+01:00,0
1,2018-01-01 01:00:00+01:00,0
2,2018-01-01 02:00:00+01:00,0
3,2018-01-01 03:00:00+01:00,0
4,2018-01-01 04:00:00+01:00,0


shape: (8760, 2)


## データの前処理

---



In [20]:
# 基本情報を確認
print(f"データのサイズ: {df.shape}")
print(f"期間: {df.index.min()} から {df.index.max()}")
print(f"推論対象の期間: {test_df.index.min()} から {test_df.index.max()}")
print(f"⽋損値の合計: {df.isna().sum().sum()}")
# ⽋損値を処理
# 時系列データなので、前⽅補間が適切
#df = df.fillna(method='ffill')
df = df.ffill()
# まだ⽋損がある場合は後⽅補間
#df = df.fillna(method='bfill')
df = df.bfill()
# ⽬的変数の確認
print("\n⽬的変数 'price_actual' の統計:")
print(df['price_actual'].describe())
# 価格の時系列プロット
plt.figure(figsize=(15, 6))
plt.plot(df.index, df['price_actual'])
plt.title('電⼒価格の推移')
plt.ylabel('価格')
plt.grid(True)
plt.tight_layout()
plt.savefig('price_time_series.png')
plt.close()
# 説明変数を準備
# item_IDはスペインの合計データのみ使⽤
df = df[df['item_ID'] == 'spain_total']
# 不要な列を削除
X = df.drop(['price_actual', 'item_ID'], axis=1)
# ⽬的変数
y = df['price_actual']

データのサイズ: (26280, 92)
期間: 2015-01-02 00:00:00+01:00 から 2017-12-31 23:00:00+01:00
推論対象の期間: 2018-01-01 00:00:00+01:00 から 2018-12-31 23:00:00+01:00
⽋損値の合計: 321

⽬的変数 'price_actual' の統計:
count    26280.000000
mean        56.028338
std         14.340356
min          9.330000
25%         47.617500
50%         55.930000
75%         65.192500
max        116.800000
Name: price_actual, dtype: float64


## 特徴量エンジニアリング

In [21]:
def create_additional_features(df):
    """
    時間ベースの特徴量とラグ特徴量を作成
    """
    df_features = df.copy()

    # インデックスがDatetimeIndexであることを確認し、必要なら変換
    if not isinstance(df_features.index, pd.DatetimeIndex) or df_features.index.tz is not None:
        df_features.index = pd.to_datetime(df_features.index, utc=True)

    # ここから既存の時間ベースの特徴量作成コード
    df_features['hour'] = df_features.index.hour
    df_features['dayofweek'] = df_features.index.dayofweek
    df_features['month'] = df_features.index.month
    df_features['year'] = df_features.index.year
    df_features['quarter'] = df_features.index.quarter
    df_features['dayofyear'] = df_features.index.dayofyear
    # ISO週番号の取得方法が古いバージョンのPandasで異なる場合があります。
    # .weekofyear は非推奨になったため、.isocalendar().week を使用するのが推奨です。
    # ただし、もし古いPandasバージョンでisocalendarが使えない場合は .weekofyear を試してください。
    try:
        df_features['weekofyear'] = df_features.index.isocalendar().week.astype(int) # ISO週番号
    except AttributeError:
        # もし isocalendar がない場合は、古い .weekofyear を試す
        print("警告: .isocalendar() が見つかりません。代わりに .weekofyear を使用します。Pandasのバージョンを確認してください。")
        df_features['weekofyear'] = df_features.index.weekofyear.astype(int)

    df_features['dayofmonth'] = df_features.index.day
    df_features['is_weekend'] = (df_features.index.dayofweek >= 5).astype(int)

    # ラグ特徴量（例: 1時間前、24時間前、48時間前、1週間前）
    # ... (既存のラグ特徴量作成コード)
    df_features['price_actual_lag_1'] = df_features['price_actual'].shift(1)
    df_features['price_actual_lag_24'] = df_features['price_actual'].shift(24)
    df_features['price_actual_lag_48'] = df_features['price_actual'].shift(48)
    df_features['price_actual_lag_168'] = df_features['price_actual'].shift(168) # 1週間前

    # 必要に応じて他の特徴量のラグも追加できます。
    # 例: df_features['consumption_lag_24'] = df_features['consumption'].shift(24)

    # 移動平均（例: 24時間の移動平均）
    df_features['price_actual_rolling_mean_24'] = df_features['price_actual'].rolling(window=24).mean()

    # 指数平滑化（例: span=24の指数平滑化）
    #df_features['price_actual_ewm_24'] = df_features['price_actual'].ewm(span=24).mean()


    # 新たに生成された特徴量によって発生する可能性のある欠損値を処理
    df_features = df_features.ffill()
    df_features = df_features.bfill()

    return df_features

## 特徴量選択と次元削減

In [22]:
# 特徴量選択（f_regressionを使って上位の特徴を選択）
def select_features_and_scale(X, y, k=30):
    """
    特徴量選択前に標準化を行い、選択された特徴量を返す
    """
    # 数値型以外のカラムを削除
    X_numeric = X.select_dtypes(include=np.number)

    # 標準化
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_numeric)
    X_scaled_df = pd.DataFrame(X_scaled, index=X_numeric.index, columns=X_numeric.columns) # DataFrameに戻す

    # もし数値型カラムがkより少ない場合はkを調整
    k = min(k, X_scaled_df.shape[1])

    selector = SelectKBest(score_func=f_regression, k=k)
    X_selected_scaled = selector.fit_transform(X_scaled_df, y)

    # 選択された特徴のインデックスを取得
    selected_indices = selector.get_support(indices=True)
    selected_features = X_scaled_df.columns[selected_indices]
    print(f"標準化後、選択された特徴量 ({k}個): {selected_features.tolist()}")

    # 選択された特徴量を DataFrame として返す（スケール済み）
    return pd.DataFrame(X_selected_scaled, index=X_numeric.index, columns=selected_features), scaler, selector

# PCAで次元削減 (選択された特徴量に対して適用)
def apply_pca_on_selected(X_selected_scaled, n_components=15):
  # X_selected_scaled はすでに StandardScaler でスケール済みを想定
  pca = PCA(n_components=n_components)
  X_pca = pca.fit_transform(X_selected_scaled)

  # 説明された分散の比率を確認
  explained_variance = pca.explained_variance_ratio_.sum()
  print(f"PCAにより{n_components}成分で{explained_variance:.2%}の分散を説明 (選択特徴量から)")
  return X_pca, pca

# 特徴量エンジニアリングを適用
df_featured = create_additional_features(df.copy())

# 特徴量エンジニアリング後のデータから説明変数と目的変数を作成
X_featured = df_featured.drop(['price_actual', 'item_ID'], axis=1)
y_featured = df_featured['price_actual']

# 特徴量選択と標準化を適用
X_selected_scaled, feature_scaler, feature_selector = select_features_and_scale(X_featured, y_featured, k=30)

# 選択された特徴量にPCAを適用
X_pca, pca = apply_pca_on_selected(X_selected_scaled, n_components=15)

# 相関ヒートマップ（選択された特徴量 - スケール済み）
# X_selected_scaled が DataFrame なのでそのまま corr() を使用
plt.figure(figsize=(15, 12))
correlation = X_selected_scaled.corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))
sns.heatmap(correlation, mask=mask, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('標準化・選択された特徴量の相関ヒートマップ')
plt.tight_layout()
plt.savefig('selected_feature_correlation.png')
plt.close()

# PCA を外す場合のデータ準備 (スケール済み・選択済みだが PCA なし)
X_no_pca = X_selected_scaled.values # numpy 配列として取得
print(f"PCA適用なしデータ形状: {X_no_pca.shape}")

標準化後、選択された特徴量 (30個): ['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'generation_other', 'total_load_actual', 'valencia_wind_speed', 'madrid_wind_speed', 'bilbao_pressure', 'bilbao_wind_speed', 'bilbao_wind_deg', 'bilbao_clouds_all', 'barcelona_wind_speed', 'seville_pressure', 'seville_wind_deg', 'hour', 'dayofweek', 'month', 'quarter', 'dayofyear', 'weekofyear', 'is_weekend', 'price_actual_lag_1', 'price_actual_lag_24', 'price_actual_lag_48', 'price_actual_lag_168', 'price_actual_rolling_mean_24']
PCAにより15成分で88.59%の分散を説明 (選択特徴量から)
PCA適用なしデータ形状: (26280, 30)


## 時系列データの準備

In [23]:
def create_sequences(X, y, seq_length=24):
    """
    時系列データのシーケンスを作成
    seq_length: 何時間前までのデータを使うか（例：24時間）
    """
    X_seq, y_seq = [], []

    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])

    return np.array(X_seq), np.array(y_seq)

# スケーリング
y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y_featured.values.reshape(-1, 1))

# データセットの分割割合
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_size = int(len(X_featured) * train_ratio)
val_size = int(len(X_featured) * val_ratio)
test_size = len(X_featured) - train_size - val_size # 残りをテストサイズとする

# -------------------------------------------------------------
# PCA適用データでの分割とシーケンス作成
# -------------------------------------------------------------
print("--- PCA適用データ ---")
X_train_pca = X_pca[:train_size]
X_val_pca = X_pca[train_size:train_size+val_size]
X_test_pca = X_pca[train_size+val_size:train_size+val_size+test_size]

y_train_scaled_pca = y_scaled[:train_size]
y_val_scaled_pca = y_scaled[train_size:train_size+val_size]
y_test_scaled_pca = y_scaled[train_size+val_size:train_size+val_size+test_size]


X_train_seq_pca, y_train_seq_pca = create_sequences(X_train_pca, y_train_scaled_pca, seq_length)
X_val_seq_pca, y_val_seq_pca = create_sequences(X_val_pca, y_val_scaled_pca, seq_length)
X_test_seq_pca, y_test_seq_pca = create_sequences(X_test_pca, y_test_scaled_pca, seq_length)

print(f"PCA適用トレーニングデータ形状: {X_train_seq_pca.shape}, {y_train_seq_pca.shape}")
print(f"PCA適用検証データ形状: {X_val_seq_pca.shape}, {y_val_seq_pca.shape}")
print(f"PCA適用テストデータ形状: {X_test_seq_pca.shape}, {y_test_seq_pca.shape}")


# -------------------------------------------------------------
# PCA適用なしデータでの分割とシーケンス作成
# X_no_pca はすでに select_features_and_scale でスケール済み
# -------------------------------------------------------------
print("\n--- PCA適用なしデータ ---")
X_train_no_pca = X_no_pca[:train_size]
X_val_no_pca = X_no_pca[train_size:train_size+val_size]
X_test_no_pca = X_no_pca[train_size+val_size:train_size+val_size+test_size]

# 目的変数は共通のスケーリング済みデータを使用
y_train_scaled_no_pca = y_scaled[:train_size]
y_val_scaled_no_pca = y_scaled[train_size:train_size+val_size]
y_test_scaled_no_pca = y_scaled[train_size+val_size:train_size+val_size+test_size]


X_train_seq_no_pca, y_train_seq_no_pca = create_sequences(X_train_no_pca, y_train_scaled_no_pca, seq_length)
X_val_seq_no_pca, y_val_seq_no_pca = create_sequences(X_val_no_pca, y_val_scaled_no_pca, seq_length)
X_test_seq_no_pca, y_test_seq_no_pca = create_sequences(X_test_no_pca, y_test_scaled_no_pca, seq_length)


print(f"PCAなしトレーニングデータ形状: {X_train_seq_no_pca.shape}, {y_train_seq_no_pca.shape}")
print(f"PCAなし検証データ形状: {X_val_seq_no_pca.shape}, {y_val_seq_no_pca.shape}")
print(f"PCAなしテストデータ形状: {X_test_seq_no_pca.shape}, {y_test_seq_no_pca.shape}")

--- PCA適用データ ---
PCA適用トレーニングデータ形状: (21000, 24, 15), (21000, 1)
PCA適用検証データ形状: (2604, 24, 15), (2604, 1)
PCA適用テストデータ形状: (2604, 24, 15), (2604, 1)

--- PCA適用なしデータ ---
PCAなしトレーニングデータ形状: (21000, 24, 30), (21000, 1)
PCAなし検証データ形状: (2604, 24, 30), (2604, 1)
PCAなしテストデータ形状: (2604, 24, 30), (2604, 1)


## モデル構築と学習

### LSTMモデル (PCA適用データ)

In [24]:
def build_lstm_model(seq_length, n_features):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(seq_length, n_features)),
        BatchNormalization(),
        Dropout(0.3),

        LSTM(128, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# LSTMモデルの構築 (PCA適用データ)
lstm_model_pca = build_lstm_model(seq_length, X_train_seq_pca.shape[2])
print("\n--- LSTMモデル (PCA適用) ---")
print(lstm_model_pca.summary())

# コールバックの設定
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    verbose=1,
    restore_best_weights=True
)

checkpoint_pca = ModelCheckpoint(
    data_path + 'lstm_model_pca.h5', # ファイル名を変更して区別
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001,
    verbose=1
)

# モデルの学習 (PCA適用データ)
print("\n--- LSTMモデル学習開始 (PCA適用) ---")
lstm_history_pca = lstm_model_pca.fit(
    X_train_seq_pca, y_train_seq_pca,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq_pca, y_val_seq_pca),
    callbacks=[early_stopping, checkpoint_pca, reduce_lr],
    verbose=1
)

# 学習履歴のプロット部分 (PCA適用)
plt.figure(figsize=(7, 5))

plt.plot(lstm_history_pca.history['loss'], label='Training Loss')
plt.plot(lstm_history_pca.history['val_loss'], label='Validation Loss')
plt.title('LSTM (PCA) - Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('lstm_training_history_loss_pca.png') # ファイル名を変更
plt.close()



--- LSTMモデル (PCA適用) ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                   │ (None, 24, 64)         │        20,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 124,225 (485.25 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 43s 87ms/step - loss: 0.0032 - val_loss: 0.0013 - learning_rate: 1.0000e-04
Epoch 71/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0032
Epoch 71: val_loss did not improve from 0.00126
329/329 ━━━━━━━━━━━━━━━━━━━━ 40s 83ms/step - loss: 0.0032 - val_loss: 0.0013 - learning_rate: 1.0000e-04
Epoch 72/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0031
Epoch 72: val_loss improved from 0.00126 to 0.00125, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 86ms/step - loss: 0.0031 - val_loss: 0.0013 - learning_rate: 1.0000e-04
Epoch 73/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0031
Epoch 73: val_loss did not improve from 0.00125
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 86ms/step - loss: 0.0031 - val_loss: 0.0014 - learning_rate: 1.0000e-04
Epoch 74/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0031
Epoch 74: val_loss did not improve from 0.00125
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 87ms/step - loss: 0.0031 - val_loss: 0.0014 - learning_rate: 1.0000e-04
Epoch 75/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0030
Epoch 75: val_loss did not improve from 0.00125
329/329 ━━━━━━━━━━━━━━━━━━━━ 40s 84ms/step - loss: 0.0030 - val_loss: 0.0014 - learning_rate: 1.0000e-04
Epoch 76/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0030
Epoch 76: val_loss did not improve from 0.00125
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 83ms/step - loss: 0.0030 - val_loss: 0.0013 - learning_rate: 1.0000e-04
Epo

329/329 ━━━━━━━━━━━━━━━━━━━━ 43s 89ms/step - loss: 0.0030 - val_loss: 0.0012 - learning_rate: 1.0000e-04
Epoch 78/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0030
Epoch 78: val_loss did not improve from 0.00122
329/329 ━━━━━━━━━━━━━━━━━━━━ 40s 85ms/step - loss: 0.0030 - val_loss: 0.0013 - learning_rate: 1.0000e-04
Epoch 79/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0030
Epoch 79: val_loss did not improve from 0.00122
329/329 ━━━━━━━━━━━━━━━━━━━━ 28s 85ms/step - loss: 0.0030 - val_loss: 0.0012 - learning_rate: 1.0000e-04
Epoch 80/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0030
Epoch 80: val_loss did not improve from 0.00122
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 85ms/step - loss: 0.0030 - val_loss: 0.0013 - learning_rate: 1.0000e-04
Epoch 81/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0030
Epoch 81: val_loss improved from 0.00122 to 0.00119, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 40s 83ms/step - loss: 0.0030 - val_loss: 0.0012 - learning_rate: 1.0000e-04
Epoch 82/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0029
Epoch 82: val_loss did not improve from 0.00119
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 82ms/step - loss: 0.0029 - val_loss: 0.0012 - learning_rate: 1.0000e-04
Epoch 83/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0029
Epoch 83: val_loss did not improve from 0.00119
329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 84ms/step - loss: 0.0029 - val_loss: 0.0013 - learning_rate: 1.0000e-04
Epoch 84/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0029
Epoch 84: val_loss did not improve from 0.00119
329/329 ━━━━━━━━━━━━━━━━━━━━ 40s 82ms/step - loss: 0.0029 - val_loss: 0.0014 - learning_rate: 1.0000e-04
Epoch 85/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0029
Epoch 85: val_loss did not improve from 0.00119
329/329 ━━━━━━━━━━━━━━━━━━━━ 43s 87ms/step - loss: 0.0029 - val_loss: 0.0015 - learning_rate: 1.0000e-04
Epo

329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 85ms/step - loss: 0.0027 - val_loss: 0.0012 - learning_rate: 1.0000e-04
Epoch 96/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0028
Epoch 96: val_loss did not improve from 0.00117
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 86ms/step - loss: 0.0028 - val_loss: 0.0015 - learning_rate: 1.0000e-04
Epoch 97/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0027
Epoch 97: val_loss did not improve from 0.00117
329/329 ━━━━━━━━━━━━━━━━━━━━ 27s 82ms/step - loss: 0.0027 - val_loss: 0.0013 - learning_rate: 1.0000e-04
Epoch 98/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0027
Epoch 98: val_loss did not improve from 0.00117
329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 85ms/step - loss: 0.0027 - val_loss: 0.0012 - learning_rate: 1.0000e-04
Epoch 99/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0026
Epoch 99: val_loss did not improve from 0.00117
329/329 ━━━━━━━━━━━━━━━━━━━━ 27s 83ms/step - loss: 0.0026 - val_loss: 0.0012 - learning_rate: 1.0000e-04
Epo

### LSTMモデル (PCA適用なし)

In [25]:
# LSTMモデルの構築 (PCA適用なしデータ)
# n_features が X_train_seq_no_pca の特徴量数になる
lstm_model_no_pca = build_lstm_model(seq_length, X_train_seq_no_pca.shape[2])
print("\n--- LSTMモデル (PCA適用なし) ---")
print(lstm_model_no_pca.summary())

checkpoint_no_pca = ModelCheckpoint(
    data_path + 'lstm_model_no_pca.h5', # ファイル名を変更して区別
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

# モデルの学習 (PCA適用なしデータ)
print("\n--- LSTMモデル学習開始 (PCA適用なし) ---")
lstm_history_no_pca = lstm_model_no_pca.fit(
    X_train_seq_no_pca, y_train_seq_no_pca,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq_no_pca, y_val_seq_no_pca),
    callbacks=[early_stopping, checkpoint_no_pca, reduce_lr], # 同じコールバックを使用
    verbose=1
)

# 学習履歴のプロット部分 (PCA適用なし)
plt.figure(figsize=(7, 5))

plt.plot(lstm_history_no_pca.history['loss'], label='Training Loss')
plt.plot(lstm_history_no_pca.history['val_loss'], label='Validation Loss')
plt.title('LSTM (No PCA) - Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('lstm_training_history_loss_no_pca.png') # ファイル名を変更
plt.close()



--- LSTMモデル (PCA適用なし) ---


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 24, 64)         │        24,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 128,065 (500.25 KB)

 Trainable params: 127,681 (498.75 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 38s 83ms/step - loss: 0.0025 - val_loss: 0.0012 - learning_rate: 2.0000e-04
Epoch 21/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0024
Epoch 21: val_loss improved from 0.00124 to 0.00116, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 86ms/step - loss: 0.0024 - val_loss: 0.0012 - learning_rate: 2.0000e-04
Epoch 22/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0023
Epoch 22: val_loss did not improve from 0.00116
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 86ms/step - loss: 0.0023 - val_loss: 0.0012 - learning_rate: 2.0000e-04
Epoch 23/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0022
Epoch 23: val_loss improved from 0.00116 to 0.00111, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 87ms/step - loss: 0.0022 - val_loss: 0.0011 - learning_rate: 2.0000e-04
Epoch 24/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0023
Epoch 24: val_loss did not improve from 0.00111
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 86ms/step - loss: 0.0023 - val_loss: 0.0011 - learning_rate: 2.0000e-04
Epoch 25/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0023
Epoch 25: val_loss improved from 0.00111 to 0.00105, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 27s 82ms/step - loss: 0.0023 - val_loss: 0.0011 - learning_rate: 2.0000e-04
Epoch 26/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0021
Epoch 26: val_loss improved from 0.00105 to 0.00101, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 85ms/step - loss: 0.0021 - val_loss: 0.0010 - learning_rate: 2.0000e-04
Epoch 27/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0021
Epoch 27: val_loss did not improve from 0.00101
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 85ms/step - loss: 0.0021 - val_loss: 0.0011 - learning_rate: 2.0000e-04
Epoch 28/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0021
Epoch 28: val_loss did not improve from 0.00101
329/329 ━━━━━━━━━━━━━━━━━━━━ 28s 84ms/step - loss: 0.0021 - val_loss: 0.0012 - learning_rate: 2.0000e-04
Epoch 29/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0020
Epoch 29: val_loss did not improve from 0.00101
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 83ms/step - loss: 0.0020 - val_loss: 0.0012 - learning_rate: 2.0000e-04
Epoch 30/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0020
Epoch 30: val_loss did not improve from 0.00101

Epoch 30: ReduceLROnPlateau reducing learning rate to 0.0001.
329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 85ms/step - 

### GRU モデル（PCA適用）

In [26]:
def build_gru_model(seq_length, n_features):
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=(seq_length, n_features)),
        BatchNormalization(),
        Dropout(0.3),

        GRU(128, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# GRUモデルの構築と学習 (PCA適用データ)
gru_model_pca = build_gru_model(seq_length, X_train_seq_pca.shape[2])
print("\n--- GRUモデル (PCA適用) ---")
print(gru_model_pca.summary())

checkpoint_gru_pca = ModelCheckpoint(
    data_path + 'gru_model_pca.h5', # ファイル名を変更して区別
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)

print("\n--- GRUモデル学習開始 (PCA適用) ---")
gru_history_pca = gru_model_pca.fit(
    X_train_seq_pca, y_train_seq_pca,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq_pca, y_val_seq_pca),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True),
        checkpoint_gru_pca,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)
    ],
    verbose=1
)

# GRU 学習履歴のプロット部分 (PCA適用)
plt.figure(figsize=(7, 5))

plt.plot(gru_history_pca.history['loss'], label='Training Loss')
plt.plot(gru_history_pca.history['val_loss'], label='Validation Loss')
plt.title('GRU (PCA) - Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('gru_training_history_loss_pca.png') # ファイル名を変更
plt.close()


--- GRUモデル (PCA適用) ---


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_2 (GRU)                     │ (None, 24, 64)         │        15,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 94,977 (371.00 KB)

 Trainable params: 94,593 (369.50 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 33s 87ms/step - loss: 0.0033 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 28/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0033
Epoch 28: val_loss did not improve from 0.00101
329/329 ━━━━━━━━━━━━━━━━━━━━ 28s 85ms/step - loss: 0.0033 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 29/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0033
Epoch 29: val_loss improved from 0.00101 to 0.00101, saving model to /content/drive/MyDrive/ML/Signate_1634/gru_model_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 28s 86ms/step - loss: 0.0033 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 30/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0032
Epoch 30: val_loss did not improve from 0.00101
329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 89ms/step - loss: 0.0032 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 31/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0032
Epoch 31: val_loss did not improve from 0.00101
329/329 ━━━━━━━━━━━━━━━━━━━━ 39s 83ms/step - loss: 0.0032 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 32/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0033
Epoch 32: val_loss did not improve from 0.00101
329/329 ━━━━━━━━━━━━━━━━━━━━ 27s 81ms/step - loss: 0.0033 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 33/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0033
Epoch 33: val_loss did not improve from 0.00101
329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 85ms/step - loss: 0.0033 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epo

329/329 ━━━━━━━━━━━━━━━━━━━━ 43s 94ms/step - loss: 0.0031 - val_loss: 9.6252e-04 - learning_rate: 1.0000e-04
Epoch 42/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0030
Epoch 42: val_loss did not improve from 0.00096
329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 87ms/step - loss: 0.0030 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 43/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0031
Epoch 43: val_loss did not improve from 0.00096
329/329 ━━━━━━━━━━━━━━━━━━━━ 28s 84ms/step - loss: 0.0031 - val_loss: 9.7059e-04 - learning_rate: 1.0000e-04
Epoch 44/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0030
Epoch 44: val_loss did not improve from 0.00096
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 84ms/step - loss: 0.0030 - val_loss: 9.9110e-04 - learning_rate: 1.0000e-04
Epoch 45/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0029
Epoch 45: val_loss did not improve from 0.00096
329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 87ms/step - loss: 0.0029 - val_loss: 0.0010 - learning_rate: 1.

329/329 ━━━━━━━━━━━━━━━━━━━━ 39s 87ms/step - loss: 0.0029 - val_loss: 9.3568e-04 - learning_rate: 1.0000e-04
Epoch 50/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0028
Epoch 50: val_loss did not improve from 0.00094
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 87ms/step - loss: 0.0028 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 51/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0028
Epoch 51: val_loss did not improve from 0.00094
329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 88ms/step - loss: 0.0028 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 52/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0028
Epoch 52: val_loss did not improve from 0.00094
329/329 ━━━━━━━━━━━━━━━━━━━━ 40s 84ms/step - loss: 0.0028 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 53/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0028
Epoch 53: val_loss did not improve from 0.00094
329/329 ━━━━━━━━━━━━━━━━━━━━ 42s 88ms/step - loss: 0.0028 - val_loss: 9.7995e-04 - learning_rate: 1.0000

329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 89ms/step - loss: 0.0027 - val_loss: 8.8717e-04 - learning_rate: 1.0000e-04
Epoch 61/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0027
Epoch 61: val_loss did not improve from 0.00089
329/329 ━━━━━━━━━━━━━━━━━━━━ 28s 86ms/step - loss: 0.0027 - val_loss: 9.1730e-04 - learning_rate: 1.0000e-04
Epoch 62/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0027
Epoch 62: val_loss did not improve from 0.00089
329/329 ━━━━━━━━━━━━━━━━━━━━ 28s 85ms/step - loss: 0.0027 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 63/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.0027
Epoch 63: val_loss did not improve from 0.00089
329/329 ━━━━━━━━━━━━━━━━━━━━ 43s 93ms/step - loss: 0.0027 - val_loss: 9.4188e-04 - learning_rate: 1.0000e-04
Epoch 64/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0027
Epoch 64: val_loss did not improve from 0.00089
329/329 ━━━━━━━━━━━━━━━━━━━━ 37s 81ms/step - loss: 0.0027 - val_loss: 9.5686e-04 - learning_rate

### GRU モデル（PCA適用なし

In [27]:
# GRUモデルの構築と学習 (PCA適用なしデータ)
gru_model_no_pca = build_gru_model(seq_length, X_train_seq_no_pca.shape[2])
print("\n--- GRUモデル (PCA適用なし) ---")
print(gru_model_no_pca.summary())

checkpoint_gru_no_pca = ModelCheckpoint(
    data_path + 'gru_model_no_pca.h5', # ファイル名を変更して区別
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)


print("\n--- GRUモデル学習開始 (PCA適用なし) ---")
gru_history_no_pca = gru_model_no_pca.fit(
    X_train_seq_no_pca, y_train_seq_no_pca,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq_no_pca, y_val_seq_no_pca),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True),
        checkpoint_gru_no_pca,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)
    ],
    verbose=1
)

# GRU 学習履歴のプロット部分 (PCA適用なし)
plt.figure(figsize=(7, 5))

plt.plot(gru_history_no_pca.history['loss'], label='Training Loss')
plt.plot(gru_history_no_pca.history['val_loss'], label='Validation Loss')
plt.title('GRU (No PCA) - Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('gru_training_history_loss_no_pca.png') # ファイル名を変更
plt.close()



--- GRUモデル (PCA適用なし) ---


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                     │ (None, 24, 64)         │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,857 (382.25 KB)

 Trainable params: 97,473 (380.75 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 88ms/step - loss: 0.0021 - val_loss: 6.1661e-04 - learning_rate: 1.0000e-04
Epoch 61/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0021
Epoch 61: val_loss did not improve from 0.00062
329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 89ms/step - loss: 0.0021 - val_loss: 6.2766e-04 - learning_rate: 1.0000e-04
Epoch 62/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0021
Epoch 62: val_loss did not improve from 0.00062
329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 89ms/step - loss: 0.0021 - val_loss: 6.4259e-04 - learning_rate: 1.0000e-04
Epoch 63/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0021
Epoch 63: val_loss did not improve from 0.00062
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 89ms/step - loss: 0.0021 - val_loss: 6.5343e-04 - learning_rate: 1.0000e-04
Epoch 64/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0021
Epoch 64: val_loss did not improve from 0.00062
329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 89ms/step - loss: 0.0021 - val_loss: 6.9628e-04 - learning_

329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 89ms/step - loss: 0.0020 - val_loss: 6.1617e-04 - learning_rate: 1.0000e-04
Epoch 68/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0019
Epoch 68: val_loss did not improve from 0.00062
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 88ms/step - loss: 0.0019 - val_loss: 6.3041e-04 - learning_rate: 1.0000e-04
Epoch 69/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0020
Epoch 69: val_loss did not improve from 0.00062
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 88ms/step - loss: 0.0020 - val_loss: 6.2292e-04 - learning_rate: 1.0000e-04
Epoch 70/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0020
Epoch 70: val_loss did not improve from 0.00062
329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 88ms/step - loss: 0.0020 - val_loss: 6.3404e-04 - learning_rate: 1.0000e-04
Epoch 71/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0020
Epoch 71: val_loss did not improve from 0.00062
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 89ms/step - loss: 0.0020 - val_loss: 6.2028e-04 - learning_

329/329 ━━━━━━━━━━━━━━━━━━━━ 44s 96ms/step - loss: 0.0019 - val_loss: 6.0676e-04 - learning_rate: 1.0000e-04
Epoch 74/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0019
Epoch 74: val_loss did not improve from 0.00061
329/329 ━━━━━━━━━━━━━━━━━━━━ 38s 87ms/step - loss: 0.0019 - val_loss: 6.3210e-04 - learning_rate: 1.0000e-04
Epoch 75/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0020
Epoch 75: val_loss improved from 0.00061 to 0.00060, saving model to /content/drive/MyDrive/ML/Signate_1634/gru_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 30s 90ms/step - loss: 0.0020 - val_loss: 6.0075e-04 - learning_rate: 1.0000e-04
Epoch 76/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0020
Epoch 76: val_loss improved from 0.00060 to 0.00059, saving model to /content/drive/MyDrive/ML/Signate_1634/gru_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 90ms/step - loss: 0.0020 - val_loss: 5.8528e-04 - learning_rate: 1.0000e-04
Epoch 77/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0019
Epoch 77: val_loss did not improve from 0.00059
329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 89ms/step - loss: 0.0019 - val_loss: 5.8846e-04 - learning_rate: 1.0000e-04
Epoch 78/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0019
Epoch 78: val_loss did not improve from 0.00059
329/329 ━━━━━━━━━━━━━━━━━━━━ 41s 89ms/step - loss: 0.0019 - val_loss: 5.9003e-04 - learning_rate: 1.0000e-04
Epoch 79/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0019
Epoch 79: val_loss did not improve from 0.00059
329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 88ms/step - loss: 0.0019 - val_loss: 5.9407e-04 - learning_rate: 1.0000e-04
Epoch 80/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0019
Epoch 80: val_loss did not improve from 0.00059
329/329 ━━━━━━━━━━━━━━━━━━━━ 29s 89ms/step - loss: 0.0019 - val_loss: 5.9854e-04 - learning_

## モデル評価と結果比較

In [29]:
def evaluate_model(model, X_test_seq, y_test_seq_scaled, y_scaler, title): # y_test_scaledをy_test_seq_scaledに変更
    """
    モデルを評価し、MAEとRMSEを計算、結果をプロットする
    """
    # 予測
    scaled_predictions = model.predict(X_test_seq)

    # スケーリングを元に戻す
    # MAE/RMSE計算とプロットのために、y_test_seq_scaledを使用する
    y_test_orig = y_scaler.inverse_transform(y_test_seq_scaled.reshape(-1, 1)).flatten() # y_test_scaledをy_test_seq_scaledに変更
    y_pred_orig = y_scaler.inverse_transform(scaled_predictions).flatten()

    # 評価指標
    mae = mean_absolute_error(y_test_orig, y_pred_orig)
    rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))

    print(f"\n--- {title} 評価結果 ---")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")

    # 予測結果と実測値の比較プロット
    plt.figure(figsize=(15, 6))
    # プロットの実測値もy_test_seq_scaledから計算する
    plt.plot(y_scaler.inverse_transform(y_test_seq_scaled.reshape(-1, 1)).flatten(), label='Actual Price') # y_test_scaledをy_test_seq_scaledに変更
    plt.plot(y_pred_orig, label='Predicted Price')
    plt.title(f'{title} - 実測値 vs 予測値')
    plt.xlabel('Time Step')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{title.replace(" ", "_")}_prediction_vs_actual.png')
    plt.close()

    return mae, rmse

# LSTM (PCA適用) の評価
# y_test_scaled_pca を y_test_seq_pca に変更して evaluate_model を呼び出す
lstm_pca_mae, lstm_pca_rmse = evaluate_model(lstm_model_pca, X_test_seq_pca, y_test_seq_pca, y_scaler, "LSTM (PCA)")

# LSTM (PCA適用なし) の評価
# y_test_scaled_no_pca を y_test_seq_no_pca に変更して evaluate_model を呼び出す
lstm_no_pca_mae, lstm_no_pca_rmse = evaluate_model(lstm_model_no_pca, X_test_seq_no_pca, y_test_seq_no_pca, y_scaler, "LSTM (No PCA)")

# GRU (PCA適用) の評価
# y_test_scaled_pca を y_test_seq_pca に変更して evaluate_model を呼び出す
gru_pca_mae, gru_pca_rmse = evaluate_model(gru_model_pca, X_test_seq_pca, y_test_seq_pca, y_scaler, "GRU (PCA)")

# GRU (PCA適用なし) の評価
# y_test_scaled_no_pca を y_test_seq_no_pca に変更して evaluate_model を呼び出す
gru_no_pca_mae, gru_no_pca_rmse = evaluate_model(gru_model_no_pca, X_test_seq_no_pca, y_test_seq_no_pca, y_scaler, "GRU (No PCA)")

# 結果の比較
print("\n--- モデル比較結果 ---")
print(f"LSTM (PCA):     MAE={lstm_pca_mae:.4f}, RMSE={lstm_pca_rmse:.4f}")
print(f"LSTM (No PCA):  MAE={lstm_no_pca_mae:.4f}, RMSE={lstm_no_pca_rmse:.4f}")
print(f"GRU (PCA):      MAE={gru_pca_mae:.4f}, RMSE={gru_pca_rmse:.4f}")
print(f"GRU (No PCA):   MAE={gru_no_pca_mae:.4f}, RMSE={gru_no_pca_rmse:.4f}")

82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step

--- LSTM (PCA) 評価結果 ---
MAE: 4.4801
RMSE: 6.2004
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step

--- LSTM (No PCA) 評価結果 ---
MAE: 3.9132
RMSE: 5.3689
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step

--- GRU (PCA) 評価結果 ---
MAE: 3.7873
RMSE: 5.1848
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step

--- GRU (No PCA) 評価結果 ---
MAE: 3.0780
RMSE: 4.3365

--- モデル比較結果 ---
LSTM (PCA):     MAE=4.4801, RMSE=6.2004
LSTM (No PCA):  MAE=3.9132, RMSE=5.3689
GRU (PCA):      MAE=3.7873, RMSE=5.1848
GRU (No PCA):   MAE=3.0780, RMSE=4.3365


## 特徴量重要度の分析

In [30]:
# 特徴量重要度の可視化（SelectKBestの結果）
def plot_feature_importance(X, y, top_n=20):
    # 数値型以外のカラムを削除
    X_numeric = X.select_dtypes(include=np.number)

    selector = SelectKBest(score_func=f_regression, k='all')
    # 数値型カラムのみを使ってfitを行う
    selector.fit(X_numeric, y)

    # 特徴量のスコアを取得
    feature_scores = pd.DataFrame({
        'Feature': X_numeric.columns, # X_numericのカラム名を使用
        'Score': selector.scores_
    })

    # スコア順にソート
    feature_scores = feature_scores.sort_values(by='Score', ascending=False)

    # 上位n個の特徴量をプロット
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Score', y='Feature', data=feature_scores.head(top_n))
    plt.title(f'上位{top_n}個の重要な特徴量')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()

    return feature_scores

# 特徴量重要度のプロット
# XがDataFrameであり、数値型以外のカラムを含んでいる可能性があるため、関数内で処理します
importance = plot_feature_importance(X_featured, y_featured, top_n=20)
print("\n上位20の重要な特徴量:")
print(importance.head(20))


上位20の重要な特徴量:
                                        Feature          Score
84                           price_actual_lag_1  362397.475584
85                          price_actual_lag_24   55455.839343
88                 price_actual_rolling_mean_24   51125.358795
87                         price_actual_lag_168   46096.371945
86                          price_actual_lag_48   29553.563560
3                   generation_fossil_hard_coal    7410.283193
1          generation_fossil_brown_coal/lignite    4315.279936
14                            total_load_actual    4191.590208
2                         generation_fossil_gas    4151.326152
5   generation_hydro_pumped_storage_consumption    2642.416186
6    generation_hydro_run_of_river_and_poundage    1905.413869
79                                      quarter    1801.790531
4                         generation_fossil_oil    1735.021199
75                                         hour    1639.636595
81                                   week

## 時系列交差検証

In [31]:
# 時系列交差検証
def time_series_cv_evaluation(X, y, seq_length=24, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    mae_scores = []
    rmse_scores = []
    fold = 1

    for train_index, test_index in tscv.split(X):
        print(f"\n評価 fold {fold}/{n_splits}")

        # データの分割
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # スケーリング
        y_scaler_fold = MinMaxScaler()
        y_train_scaled = y_scaler_fold.fit_transform(y_train_fold.reshape(-1, 1))
        y_test_scaled = y_scaler_fold.transform(y_test_fold.reshape(-1, 1))

        # シーケンスデータの作成
        X_train_seq_fold, y_train_seq_fold = create_sequences(X_train_fold, y_train_scaled, seq_length)
        X_test_seq_fold, y_test_seq_fold = create_sequences(X_test_fold, y_test_scaled, seq_length)

        if len(X_train_seq_fold) == 0 or len(X_test_seq_fold) == 0:
            print("シーケンスデータが空です")
            continue

        # モデル構築と学習
        model = build_lstm_model(seq_length, X_train_seq_fold.shape[2])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

        model.fit(
            X_train_seq_fold, y_train_seq_fold,
            epochs=50,
            batch_size=64,
            validation_split=0.2,
            callbacks=[early_stopping],
            verbose=0
        )

        # 評価
        y_pred_fold = model.predict(X_test_seq_fold)

        # スケーリングを元に戻す
        y_test_orig_fold = y_scaler_fold.inverse_transform(y_test_seq_fold.reshape(-1, 1)).flatten()
        y_pred_orig_fold = y_scaler_fold.inverse_transform(y_pred_fold).flatten()

        # 評価指標の計算
        mae = mean_absolute_error(y_test_orig_fold, y_pred_orig_fold)
        rmse = np.sqrt(mean_squared_error(y_test_orig_fold, y_pred_orig_fold))

        mae_scores.append(mae)
        rmse_scores.append(rmse)

        print(f"Fold {fold} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
        fold += 1

    # 結果をまとめる
    print("\n時系列交差検証の結果:")
    print(f"平均 MAE: {np.mean(mae_scores):.2f} (±{np.std(mae_scores):.2f})")
    print(f"平均 RMSE: {np.mean(rmse_scores):.2f} (±{np.std(rmse_scores):.2f})")

    return mae_scores, rmse_scores

# 時系列交差検証の実行
mae_cv, rmse_cv = time_series_cv_evaluation(X_pca, y_scaled, seq_length=24, n_splits=3)


評価 fold 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step
Fold 1 - MAE: 0.09, RMSE: 0.12

評価 fold 2/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step
Fold 2 - MAE: 0.08, RMSE: 0.10

評価 fold 3/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step
Fold 3 - MAE: 0.04, RMSE: 0.06

時系列交差検証の結果:
平均 MAE: 0.07 (±0.02)
平均 RMSE: 0.09 (±0.03)


## モデルとパイプラインの保存

In [32]:
import pickle

# モデル保存のためのパイプラインを作成
pipeline = {
    'pca': pca,
    'pca_scaler': pca_scaler,
    'y_scaler': y_scaler,
    'selected_features': X_selected.columns.tolist(),
    'seq_length': seq_length
}

# パイプラインを保存
pipeline_save_path = data_path + 'price_prediction_pipeline.pkl'
with open(pipeline_save_path, 'wb') as f:
    pickle.dump(pipeline, f)

# 予測パイプラインの使用例
def predict_price(model, new_data, pipeline):
    """
    新しいデータで価格を予測する

    Parameters:
    - model: 訓練済みモデル
    - new_data: 予測したいデータ（必要な特徴量を含むDataFrame）
    - pipeline: 前処理パイプライン

    Returns:
    - predicted_price: 予測された価格
    """
    # パイプラインから必要な要素を取得
    pca = pipeline['pca']
    pca_scaler = pipeline['pca_scaler']
    y_scaler = pipeline['y_scaler']
    selected_features = pipeline['selected_features']
    seq_length = pipeline['seq_length']

    # 必要な特徴量のみを選択
    selected_data = new_data[selected_features]

    # スケーリングとPCA変換
    scaled_data = pca_scaler.transform(selected_data)
    pca_data = pca.transform(scaled_data)

    # シーケンスデータの作成（最新のseq_length分のデータを使用）
    if len(pca_data) >= seq_length:
        sequence = pca_data[-seq_length:].reshape(1, seq_length, -1)

        # 予測
        scaled_prediction = model.predict(sequence)

        # スケーリングを元に戻す
        prediction = y_scaler.inverse_transform(scaled_prediction)[0, 0]

        return prediction
    else:
        raise ValueError(f"入力データが短すぎます。少なくとも{seq_length}点必要です。")

print("\nモデルと前処理パイプラインを保存しました。")
print("新しいデータに対する予測は 'predict_price' 関数を使用して行えます。")


モデルと前処理パイプラインを保存しました。
新しいデータに対する予測は 'predict_price' 関数を使用して行えます。


### 最適モデルの決定と最終予測

In [34]:

# モデルの評価結果を格納
evaluation_results = {
    "LSTM (PCA)": {"mae": lstm_pca_mae, "rmse": lstm_pca_rmse, "model": lstm_model_pca, "data_type": "pca"},
    "LSTM (No PCA)": {"mae": lstm_no_pca_mae, "rmse": lstm_no_pca_rmse, "model": lstm_model_no_pca, "data_type": "no_pca"},
    "GRU (PCA)": {"mae": gru_pca_mae, "rmse": gru_pca_rmse, "model": gru_model_pca, "data_type": "pca"},
    "GRU (No PCA)": {"mae": gru_no_pca_mae, "rmse": gru_no_pca_rmse, "model": gru_model_no_pca, "data_type": "no_pca"}
}

print("\n--- モデル比較結果 ---")
for name, metrics in evaluation_results.items():
    print(f"{name}: MAE={metrics['mae']:.4f}, RMSE={metrics['rmse']:.4f}")

# 最適なモデルを選択（ここではMAEが最小のモデルとする）
best_model_name = min(evaluation_results, key=lambda k: evaluation_results[k]['mae'])
best_model_info = evaluation_results[best_model_name]
best_model = best_model_info['model']
best_data_type = best_model_info['data_type']

print(f"\n最適なモデルは: {best_model_name} (MAE={best_model_info['mae']:.4f})")

# 最終予測に使用する前処理パイプラインを準備
# 学習時に使用したスケーラーとPCAオブジェクトが必要
# Assuming feature_scaler, pca, y_scaler are available from previous steps

final_pca = None
final_feature_scaler = feature_scaler # 特徴量選択前の標準化に使用したscaler
final_y_scaler = y_scaler

if best_data_type == 'pca':
    final_pca = pca # 最適モデルがPCAを使用した場合のみPCAオブジェクトを使用
    print("最終予測にPCAを使用します。")
else:
    print("最終予測にPCAは使用しません。")


# test_dfに対する最終推論
# test_IDが'spain_total'のデータのみを選択
test_df_spain = test_df[test_df['item_ID'] == 'spain_total'].copy()

# 欠損値処理 (trainと同じ処理を適用)
test_df_spain = test_df_spain.ffill()
test_df_spain = test_df_spain.bfill()

# 'item_ID' 列を削除
test_df_spain = test_df_spain.drop('item_ID', axis=1)

# テストデータに特徴量エンジニアリングを適用（学習データを使ってラグを計算）
# create_additional_features_combined 関数が train_df を必要とするため渡す
if 'df' in locals() or 'df' in globals():
    test_df_featured = create_additional_features_combined(
        test_df_spain.copy(),
        is_test=True,
        train_df=df.copy() # 学習データを使用
    )
    print(f"\nテストデータの特徴量エンジニアリング完了。形状: {test_df_featured.shape}")

    # NaN値の最終確認と処理
    final_nan_count_test = test_df_featured.isnull().sum().sum()
    if final_nan_count_test > 0:
        print(f"警告: テストデータの特徴量エンジニアリング後にNaN値が {final_nan_count_test}個 あります。0で埋めます。")
        test_df_featured = test_df_featured.fillna(0)
else:
    print("エラー: 学習データ (df) が見つかりません。テストデータの特徴量エンジニアリングを実行できません。")
    test_df_featured = None


if test_df_featured is not None:
    try:
        # 学習時に選択された特徴量のみを抽出
        # loaded_selected_features は前回のパイプラインロード部分で取得
        # ここでは feature_selector から再取得する
        selected_features_for_inference = feature_selector.get_support(indices=True)
        selected_feature_names = X_featured.select_dtypes(include=np.number).columns[selected_features_for_inference].tolist()

        test_data_aligned = test_df_featured[selected_feature_names]
        print(f"テストデータ ({test_data_aligned.shape[1]}個の特徴量) のカラム順序を合わせました。")

        # テストデータに前処理を適用 (標準化)
        test_data_scaled = final_feature_scaler.transform(test_data_aligned)
        test_data_processed = test_data_scaled

        # PCA適用が必要な場合のみPCAを適用
        if final_pca is not None:
            test_data_processed = final_pca.transform(test_data_scaled)
            print(f"テストデータにPCAを適用しました。形状: {test_data_processed.shape}")
        else:
            print(f"テストデータにPCAは適用しません。形状: {test_data_processed.shape}")


        # 推論用シーケンスの作成
        # 学習データの最終部分 + テストデータ全体を使ってシーケンスを作成する必要がある
        # 結合データが必要
        if best_data_type == 'pca':
             # 学習データのPCA適用済みデータ + テストデータのPCA適用済みデータ
             if 'X_pca' in locals() or 'X_pca' in globals():
                 combined_data_for_inference = np.concatenate([X_pca, test_data_processed])
                 print(f"推論用に学習(PCA済)とテスト(PCA済)データを結合。形状: {combined_data_for_inference.shape}")
             else:
                 print("エラー: 学習データのPCA結果 (X_pca) が見つかりません。推論できません。")
                 combined_data_for_inference = None

        else: # best_data_type == 'no_pca'
             # 学習データのPCAなしデータ + テストデータのPCAなしデータ
             if 'X_no_pca' in locals() or 'X_no_pca' in globals():
                  # X_no_pca は既に scaled/selected 済み
                  combined_data_for_inference = np.concatenate([X_no_pca, test_data_processed])
                  print(f"推論用に学習(PCAなし)とテスト(PCAなし)データを結合。形状: {combined_data_for_inference.shape}")
             else:
                 print("エラー: 学習データのPCAなしデータ (X_no_pca) が見つかりません。推論できません。")
                 combined_data_for_inference = None


        X_inference = []
        # 推論はテストデータの最初から開始
        # シーケンスの開始位置を考慮して、結合データからの開始インデックスを決定
        start_index_combined = len(df) - loaded_seq_length # ラグ特徴量などを考慮した開始点
        if best_data_type == 'pca':
             start_index_combined = len(X_pca) - loaded_seq_length
             # 学習データとテストデータを結合したデータセットに対してシーケンスを作成
             # テストデータの最初のタイムステップに対応するシーケンスが必要
             # combined_data_for_inference は (len(X_pca) + len(test_data_processed)) の長さ
             # テストデータの最初のタイムステップはインデックス len(X_pca)
             # その予測に必要なシーケンスは combined_data_for_inference[len(X_pca) - loaded_seq_length : len(X_pca)] から始まる
             inference_start_idx_in_combined = len(X_pca) - loaded_seq_length
        else: # best_data_type == 'no_pca'
             start_index_combined = len(X_no_pca) - loaded_seq_length
             inference_start_idx_in_combined = len(X_no_pca) - loaded_seq_length


        if combined_data_for_inference is not None:
             # テストデータの長さ分の予測が必要
             num_test_timesteps = len(test_data_processed)

             for i in range(num_test_timesteps):
                 seq_start = inference_start_idx_in_combined + i
                 seq_end = seq_start + loaded_seq_length

                 if seq_start >= 0 and seq_end <= len(combined_data_for_inference):
                      input_sequence = combined_data_for_inference[seq_start:seq_end]
                      X_inference.append(input_sequence)
                 else:
                      # シーケンスが結合データの範囲外になる場合はスキップまたは適切な処理
                      print(f"警告: シーケンス範囲外のためスキップ (start={seq_start}, end={seq_end}, combined_len={len(combined_data_for_inference)})")


        if X_inference:
            X_inference = np.array(X_inference)
            print(f"推論用シーケンスデータ形状: {X_inference.shape}")

            # 予測実行
            scaled_predictions = best_model.predict(X_inference)
            predictions = final_y_scaler.inverse_transform(scaled_predictions).flatten()

            # 提出ファイル作成
            # 予測されたのはテストデータの seq_length 以降の部分に対応する
            # 推論用シーケンス作成時のループ回数と予測結果の長さを確認
            if len(predictions) == len(test_df_spain) - loaded_seq_length:
                # テストデータの対応するタイムスタンプを取得
                # test_df_spain のインデックスで、シーケンス長だけずらす
                valid_test_indices = test_df_spain.index[loaded_seq_length:]

                submission_df = pd.DataFrame({
                    'id': [ts.strftime('%Y-%m-%d %H:%M:%S') + '_spain_total' for ts in valid_test_indices],
                    'price_actual': predictions
                })

                submission_path = data_path + 'submission.csv'
                submission_df.to_csv(submission_path, header=False, index=False)

                print(f"\n最終推論結果を '{submission_path}' に保存しました。")
                print("提出ファイルの先頭:")
                display(pd.read_csv(submission_path, header=None).head())

            else:
                print(f"エラー: 予測結果の長さ ({len(predictions)}) がテストデータの予測対象期間の長さ ({len(test_df_spain) - loaded_seq_length}) と一致しません。")
                print(f"テストデータの長さ: {len(test_df_spain)}")
                print(f"シーケンス長: {loaded_seq_length}")
                print(f"予測される長さ: {len(test_df_spain) - loaded_seq_length}")


        else:
            print("エラー: 最終予測のための入力シーケンスが作成できませんでした。")

    except KeyError as e:
        print(f"エラー: 特徴量選択で問題が発生しました。テストデータに必要な特徴量が見つかりません: {e}")
    except Exception as e:
        print(f"最終予測の実行中にエラーが発生しました: {e}")

else:
    print("テストデータの特徴量エンジニアリングが正常に完了しなかったため、推論を実行できません。")
# %%


--- モデル比較結果 ---
LSTM (PCA): MAE=4.4801, RMSE=6.2004
LSTM (No PCA): MAE=3.9132, RMSE=5.3689
GRU (PCA): MAE=3.7873, RMSE=5.1848
GRU (No PCA): MAE=3.0780, RMSE=4.3365

最適なモデルは: GRU (No PCA) (MAE=3.0780)
最終予測にPCAは使用しません。

テストデータの特徴量エンジニアリング完了。形状: (8760, 105)
テストデータ (30個の特徴量) のカラム順序を合わせました。
最終予測の実行中にエラーが発生しました: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- barcelona_clouds_all
- barcelona_humidity
- barcelona_pressure
- barcelona_rain_1h
- barcelona_rain_3h
- ...



<ipython-input-33-ca4bec39412f>:72: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_features = df_features.ffill()


## 最終提出データの作成

In [33]:
import pandas as pd
import numpy as np
import os
import pickle
import tensorflow as tf

# test_dfに対する推論
# test_IDが'spain_total'のデータのみを選択
test_df_spain = test_df[test_df['item_ID'] == 'spain_total'].copy()

# 欠損値処理 (trainと同じ処理を適用)
test_df_spain = test_df_spain.ffill()
test_df_spain = test_df_spain.bfill()

# 'item_ID' 列を削除
test_df_spain = test_df_spain.drop('item_ID', axis=1)

def create_additional_features_combined(df, is_test=False, train_df=None):
    """
    時間ベースの特徴量とラグ特徴量を作成。
    テストデータの場合は学習データと結合してラグ特徴量を正しく計算する。
    """
    df_features = df.copy()

    # インデックスがDatetimeIndexであることを確認し、必要なら変換
    if not isinstance(df_features.index, pd.DatetimeIndex) or df_features.index.tz is not None:
        df_features.index = pd.to_datetime(df_features.index, utc=True)

    # 時間ベースの特徴量
    df_features['hour'] = df_features.index.hour
    df_features['dayofweek'] = df_features.index.dayofweek
    df_features['month'] = df_features.index.month
    df_features['year'] = df_features.index.year
    df_features['quarter'] = df_features.index.quarter
    df_features['dayofyear'] = df_features.index.dayofyear
    try:
        df_features['weekofyear'] = df_features.index.isocalendar().week.astype(int)
    except AttributeError:
        df_features['weekofyear'] = df_features.index.weekofyear.astype(int)
    df_features['dayofmonth'] = df_features.index.day
    df_features['is_weekend'] = (df_features.index.dayofweek >= 5).astype(int)

    # ラグ特徴量の処理
    if is_test and train_df is not None:
        # テストデータの場合、学習データと結合してラグ特徴量を計算
        # 学習データの最後の部分とテストデータを結合
        combined_df = pd.concat([train_df, df_features], axis=0)

        # 結合データに対してラグ特徴量を計算
        combined_df['price_actual_lag_1'] = combined_df['price_actual'].shift(1)
        combined_df['price_actual_lag_24'] = combined_df['price_actual'].shift(24)
        combined_df['price_actual_lag_48'] = combined_df['price_actual'].shift(48)
        combined_df['price_actual_lag_168'] = combined_df['price_actual'].shift(168)
        combined_df['price_actual_rolling_mean_24'] = combined_df['price_actual'].rolling(window=24).mean()

        # テストデータ部分のみを抽出
        df_features = combined_df.loc[df_features.index]

        # price_actual列を削除（テストデータには存在しないため）
        if 'price_actual' in df_features.columns:
            df_features = df_features.drop('price_actual', axis=1)

    elif not is_test and 'price_actual' in df_features.columns:
        # 学習データの場合
        df_features['price_actual_lag_1'] = df_features['price_actual'].shift(1)
        df_features['price_actual_lag_24'] = df_features['price_actual'].shift(24)
        df_features['price_actual_lag_48'] = df_features['price_actual'].shift(48)
        df_features['price_actual_lag_168'] = df_features['price_actual'].shift(168)
        df_features['price_actual_rolling_mean_24'] = df_features['price_actual'].rolling(window=24).mean()

    # 欠損値処理
    df_features = df_features.ffill()
    df_features = df_features.bfill()

    # まだNaNが残っている場合は0で埋める
    df_features = df_features.fillna(0)

    return df_features

# 保存したパイプラインを読み込み
pipeline_load_path = data_path + 'price_prediction_pipeline.pkl'
loaded_pipeline = None
loaded_selected_features = None

if os.path.exists(pipeline_load_path):
    try:
        with open(pipeline_load_path, 'rb') as f:
            loaded_pipeline = pickle.load(f)

        loaded_pca = loaded_pipeline['pca']
        loaded_pca_scaler = loaded_pipeline['pca_scaler']
        loaded_y_scaler = loaded_pipeline['y_scaler']
        loaded_selected_features = loaded_pipeline['selected_features']
        loaded_seq_length = loaded_pipeline['seq_length']

        print(f"\nパイプラインをロードしました。学習時に選択された特徴量 ({len(loaded_selected_features)}個): {loaded_selected_features}")

    except Exception as e:
        print(f"パイプラインのロードに失敗しました: {e}")
        loaded_pipeline = None
else:
    print(f"エラー: パイプラインファイル '{pipeline_load_path}' が見つかりません。")

if loaded_pipeline is not None and loaded_selected_features is not None:
    # 学習データの特徴量エンジニアリング済みデータが必要
    # dfに対して同じ特徴量エンジニアリングを適用
    if 'df' in locals() or 'df' in globals():
        # 学習データに特徴量エンジニアリングを適用（price_actualラグを含む）
        df_featured_train = create_additional_features_combined(df.copy(), is_test=False)

        # テストデータに特徴量エンジニアリングを適用（学習データを使ってラグを計算）
        test_df_featured = create_additional_features_combined(
            test_df_spain.copy(),
            is_test=True,
            train_df=df.copy()
        )

        print(f"テストデータの特徴量エンジニアリング完了。形状: {test_df_featured.shape}")

        # NaN値の確認
        nan_count = test_df_featured.isnull().sum().sum()
        print(f"テストデータのNaN値の総数: {nan_count}")

        if nan_count > 0:
            print("NaN値が残っているカラム:")
            nan_columns = test_df_featured.isnull().sum()
            print(nan_columns[nan_columns > 0])

            # 残りのNaN値を0で埋める
            test_df_featured = test_df_featured.fillna(0)
            print("残りのNaN値を0で埋めました。")

        try:
            # 学習時に選択された特徴量のみを抽出
            test_data_aligned = test_df_featured[loaded_selected_features]
            print(f"テストデータ ({test_data_aligned.shape[1]}個の特徴量) のカラム順序を合わせました。")

            # 最終的なNaN確認
            final_nan_count = test_data_aligned.isnull().sum().sum()
            print(f"最終的なNaN値の総数: {final_nan_count}")

            if final_nan_count > 0:
                print("警告: まだNaN値が残っています。さらに処理します。")
                test_data_aligned = test_data_aligned.fillna(method='ffill').fillna(method='bfill').fillna(0)

        except KeyError as e:
            print(f"エラー: 学習時に選択された特徴量 '{e}' がテストデータに見つかりません。")
            test_data_aligned = None

        if test_data_aligned is not None:
            # モデルをロード
            lstm_model_path = data_path + 'lstm_model.h5'
            try:
                loaded_model = tf.keras.models.load_model(
                    lstm_model_path,
                    custom_objects={'mse': tf.keras.losses.MeanSquaredError()}
                )
                print("モデルを正常にロードしました。")

                # テストデータに前処理を適用
                test_data_scaled = loaded_pca_scaler.transform(test_data_aligned)
                test_data_pca = loaded_pca.transform(test_data_scaled)
                print(f"テストデータにスケーリングとPCAを適用しました。形状: {test_data_pca.shape}")

                # 学習データのPCA結果と結合
                if 'X_pca' in locals() or 'X_pca' in globals():
                    combined_data_pca = np.concatenate([X_pca, test_data_pca])
                    print(f"データを結合しました。形状: {combined_data_pca.shape}")

                    # 推論用シーケンスの作成
                    X_inference = []
                    start_index_test = len(X_pca)

                    for i in range(len(test_data_pca)):
                        seq_start = start_index_test + i - loaded_seq_length
                        seq_end = start_index_test + i

                        if seq_start >= 0:
                            input_sequence = combined_data_pca[seq_start:seq_end]
                            X_inference.append(input_sequence)

                    if X_inference:
                        X_inference = np.array(X_inference)
                        print(f"推論用シーケンスデータ形状: {X_inference.shape}")

                        # 予測実行
                        scaled_predictions = loaded_model.predict(X_inference)
                        predictions = loaded_y_scaler.inverse_transform(scaled_predictions).flatten()

                        # 提出ファイル作成
                        valid_test_indices = test_df_spain.index[len(test_df_spain) - len(predictions):]
                        submission_df = pd.DataFrame({
                            'id': [ts.strftime('%Y-%m-%d %H:%M:%S') + '_spain_total' for ts in valid_test_indices],
                            'price_actual': predictions
                        })

                        submission_path = data_path + 'submission.csv'
                        submission_df.to_csv(submission_path, header=False, index=False)

                        print(f"\n推論結果を '{submission_path}' に保存しました。")
                        print("提出ファイルの先頭:")
                        display(pd.read_csv(submission_path, header=None).head())

                    else:
                        print("エラー: 予測のための入力シーケンスが作成できませんでした。")
                else:
                    print("エラー: 学習データのPCA結果 (X_pca) が見つかりません。")

            except Exception as e:
                print(f"モデルのロードまたは推論に失敗しました: {e}")
    else:
        print("エラー: 学習データ (df) が見つかりません。")
else:
    print("パイプラインがロードされなかったため、推論を実行できません。")


パイプラインをロードしました。学習時に選択された特徴量 (30個): ['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'generation_other', 'total_load_actual', 'valencia_wind_speed', 'madrid_wind_speed', 'bilbao_pressure', 'bilbao_wind_speed', 'bilbao_wind_deg', 'bilbao_clouds_all', 'barcelona_wind_speed', 'seville_pressure', 'seville_wind_deg', 'hour', 'dayofweek', 'month', 'quarter', 'dayofyear', 'weekofyear', 'is_weekend', 'price_actual_lag_1', 'price_actual_lag_24', 'price_actual_lag_48', 'price_actual_lag_168', 'price_actual_rolling_mean_24']


<ipython-input-33-ca4bec39412f>:72: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_features = df_features.ffill()


テストデータの特徴量エンジニアリング完了。形状: (8760, 105)
テストデータのNaN値の総数: 0
テストデータ (30個の特徴量) のカラム順序を合わせました。
最終的なNaN値の総数: 0


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


モデルを正常にロードしました。
テストデータにスケーリングとPCAを適用しました。形状: (8760, 15)
データを結合しました。形状: (35040, 15)
推論用シーケンスデータ形状: (8760, 24, 15)
274/274 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step

推論結果を '/content/drive/MyDrive/ML/Signate_1634/submission.csv' に保存しました。
提出ファイルの先頭:


,0,1
0,2018-01-01 00:00:00_spain_total,59.314354
1,2018-01-01 01:00:00_spain_total,58.704500
2,2018-01-01 02:00:00_spain_total,58.757313
3,2018-01-01 03:00:00_spain_total,60.397420
4,2018-01-01 04:00:00_spain_total,61.126390


### 最終メモ
この修正により、PCAによる次元削減がモデル性能にどのような影響を与えるかを定量的に比較できるようになります。

次に進めるべきステップとしては、ハイパーパラメータ検索の導入や、時系列交差検証のロジック一本化・ログ強化が考えられます。どちらに進みますか？ または、今回のコードについてさらに詳細な説明や修正が必要な点があればお知らせください。